In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
xgb.set_config(verbosity=0)
from sklearn.feature_selection import RFECV
import random
import uniprot
pd.set_option('display.max_columns', None)

In [14]:
df = pd.read_csv("dataset\CSF_Proteomics_ADNI.csv")

#replace categorical feature with index labeling
df['binary_class'].replace({'stable':0,'decliner':1},inplace=True)
df['three_class'].replace({'slowDecline':0,'rapidDecline':1,'stable':2},inplace=True)

#differentiate other categorical features from the numerical ones
pheno = df.loc[:,'RID':'VISCODE']
data = df.loc[:,'A1AT.AVLTIDEK':'VTDB.VPTADLEDVLPLAEDITNILSK']

#check head of numerical sub-dataset
data.head()

,A1AT.AVLTIDEK,A1AT.LSITGTYDLK,A1AT.SVLGQLGITK,A1BG.NGVAQEPVHLDSPAIK,A1BG.SGLSTGWTQLSK,A2GL.DLLLPQPDLR,A2GL.VAAGAFQGLR,A4.LVFFAEDVGSNK,A4.THPHFVIPYR,A4.WYFDVTEGK,AACT.ADLSGITGAR,AACT.EIGELYLPK,AACT.NLAVSQVVHK,AATC.IVASTLSNPELFEEWTGNVK,AATC.LALGDDSPALK,AATC.NLDYVATSIHEAVTK,AATM.FVTVQTISGTGALR,AFAM.DADPDTFFAK,AFAM.FLVNLVK,AFAM.LPNNVLQEK,ALDOA.ALQASALK,ALDOA.QLLLTADDR,AMBP.AFIQLWAFDAVK,AMBP.ETLLQDFR,AMBP.FLYHK,AMD.IPVDEEAFVIDFKPR,AMD.IVQFSPSGK,AMD.NGQWTLIGR,APLP2.HYQHVLAVDPEK,APLP2.WYFDLSK,APOB.IAELSATAQEIIK,APOB.SVSLPSLDPASAK,APOB.TGISPLALIK,APOD.VLNQELR,APOE.AATVGSLAGQPLQER,APOE.CLAVYQAGAR,APOE.LAVYQAGAR,APOE.LGADMEDVR,APOE.LGPLVEQGR,B2MG.VEHSDLSFSK,B2MG.VNHVTLSQPK,B3GN1.EPGEFALLR,B3GN1.TALASGGVLDASGDYR,B3GN1.YEAAVPDPR,BACE1.SIVDSGTTNLR,BASP1.ETPAATEAPSSTPK,BTD.LSSGLVTAALYGR,BTD.SHLIIAQVAK,C1QB.LEQGENVFLQATDK,C1QB.VPGLYYFTYHASSR,CA2D1.FVVTDGGITR,CA2D1.IKPVFIEDANFGR,CA2D1.TASGVNQLVDIYEK,CAD13.DIQGSLQDIFK,CAD13.INENTGSVSVTR,CAD13.YEVSSPYFK,CADM3.EGSVPPLK,CADM3.GNPVPQQYLWEK,CADM3.SLVTVLGIPQKPIITGYK,CAH1.VLDALQAIK,CAH1.YSSLAEAASK,CATA.LFAYPDTHR,CATD.LVDQNIFSFYLSR,CATD.VSTLPAITLK,CATD.YSQAVPAVTEGPIPEVLK,CATL1.VFQEPLFYEAPR,CCKN.AHLGALLAR,CCKN.NLQNLDPSHR,CD14.AFPALTSLDLSDNPGLGER,CD14.FPAIQNLALR,CD14.SWLAELQQWLKPGLK,CD59.AGLQVYNK,CERU.IYHSHIDAPK,CERU.NNEGTYYSPNYNPQSR,CFAB.DAQYAPGYDK,CFAB.VSEADSSNADWVTK,CFAB.YGLVTYATYPK,CH3L1.ILGQQVPYATK,CH3L1.SFTLASSETGVGAPISGPGIPGR,CH3L1.VTIDSSYDIAK,CLUS.IDSLLENDR,CLUS.SGSGLVGR,CLUS.VTTVASHTSDSDVPSGVTEVVVK,CMGA.EDSLEAGLPLQVR,CMGA.SEALAVDGAGKPGAEEAQDPEGK,CMGA.SGEATDGARPQALPEPMQESK,CMGA.SGELEQEEER,CMGA.YPGPQAEGDSEGLSQGLVDR,CNDP1.ALEQDLPVNIK,CNDP1.VFQYIDLHQDEFVQTLK,CNDP1.WNYIEGTK,CNTN1.DGEYVVEVR,CNTN1.TTKPYPADIVVQFK,CNTN2.IIVQAQPEWLK,CNTN2.TTGPGGDGIPAEVHIVR,CNTN2.VIASNILGTGEPSGPSSK,CNTP2.HELQHPIIAR,CNTP2.VDNAPDQQNSHPDLAQEEIR,CNTP2.YSSSDWVTQYR,CO2.DFHINLFR,CO2.HAIILLTDGK,CO2.SSGQWQTPGATR,CO3.IHWESASLLR,CO3.LSINTHPSQKPLSITVR,CO3.TELRPGETLNVNFLLR,CO3.TGLQEVEVK,CO3.VPVAVQGEDTVQSLTQGDGVAK,CO4A.DHAVDLIQK,CO4A.GSFEFPVGDAVSK,CO4A.LGQYASPTAK,CO4A.NVNFQK,CO4A.VLSLAQEQVGGSPEK,CO4A.VTASDPLDTLGSEGALSPGGVASLLR,CO5.DINYVNPVIK,CO5.TLLPVSKPEIR,CO5.VFQFLEK,CO6.ALNHLPLEYNSALYSR,CO6.SEYGAALAWEK,CO8B.IPGIFELGISSQSDR,CO8B.SDLEVAHYK,CO8B.YEFILK,COCH.GVISNSGGPVR,CRP.ESDTSYVSLK,CSTN1.GNLAGLTLR,CSTN1.IHGQNVPFDAVVVDK,CSTN1.IPDGVVSVSPK,CSTN3.ATGEGLIR,CSTN3.ESLLLDTTSLQQR,CUTA.TQSSLVPALTDFVR,CYTC.ALDFAVGEYNK,DAG1.GVHYISVSATR,DAG1.LVPVVNNR,DAG1.VTIPTDLIASSGDIIK,DIAC.ATYIQNYR,ENOG.GNPTVEVDLYTAK,ENOG.LGAEVYHTLK,ENPP2.SYPEILTLK,ENPP2.WWGGQPLWITATK,EXTL2.VIVVWNNIGEK,FABPH.SIVTLDGGK,FABPH.SLGVGFATR,FAM3C.GINVALANGK,FAM3C.SALDTAAR,FAM3C.SPFEQHIK,FAM3C.TGEVLDTK,FBLN1.AITPPHPASQANIIFDITEGNLR,FBLN1.IIEVEEEQEDPYLNDR,FBLN1.TGYYFDGISR,FBLN3.IPSNPSHR,FBLN3.LTIIVGPFSF,FBLN3.SGNENGEFYLR,FETUA.AHYDLR,FETUA.FSVVYAK,FETUA.HTLNQIDEVK,FMOD.YLPFVPSR,GFAP.ALAAELNQLR,GOLM1.DQLVIPDGQEEEQEAAGEGR,GOLM1.QQLQALSEPQPR,GRIA4.EYPGSETPPK,GRIA4.LQNILEQIVSVGK,GRIA4.NTDQEYTAFR,HBA.FLASVSTVLTSK,HBA.TYFPHFDLSHGSAQVK,HBA.VGAHAGEYGAEALER,HBB.EFTPPVQAAYQK,HBB.SAVTALWGK,HBB.VNVDEVGGEALGR,HEMO.NFPSPVDAAFR,HEMO.QGHNSVFLIK,HEMO.SGAQATWTELPWPHEK,I18BP.LWEGSTSR,IBP2.HGLYNLK,IBP2.LIQGAPTIR,IGSF8.DTQFSYAVFK,IGSF8.LQGDAVVLK,IGSF8.VVAGEVQVQR,ITIH1.EVAFDLEIPK,ITIH1.QYYEGSEIVVAGR,ITIH5.SYLEITPSR,KAIN.FYYLIASETPGK,KAIN.LGFTDLFSK,KAIN.VGSALFLSHNLK,KAIN.WADLSGITK,KLK10.ALQLPYR,KLK11.LPHTLR,KLK6.ESSQEQSSVVR,KLK6.LSELIQPLPLER,KLK6.YTNWIQK,KNG1.DIPTNSPELEETLTHTITK,KNG1.QVVAGLNFR,KNG1.TVGSDTFYSFK,KPYM.LDIDSPPITAR,L1CAM.AQLLVVGSPGPVPR,L1CAM.LVLSDLHLLTQSQVR,L1CAM.WRPVDLAQVK,LAMB2.AQGIAQGAIR,LPHN1.LVVSQLNPYTLR,LPHN1.SGETVINTANYHDTSPYR,LRC4B.HLEILQLSK,LRC4B.LTTVPTQAFEYLSK,LTBP2.EQDAPVAGLQPVER,MIME.ESAYLYAR,MIME.ETVIIPNEK,MIME.LEGNPIVLGK,MOG.VVHLYR,MUC18.EVTVPVFYPTEK,MUC18.GATLALTQVTPQDER,NBL1.LALFPDK,NCAM1.AGEQDATIHLK,NCAM1.GLGEISAASEFK,NCAM2.ASGSPEPAISWFR,NCAM2.IIELSQTTAK,NCAN.APVLELEK,NCAN.LSSAIIAAPR,NEGR1.SSIIFAGGDK,NEGR1.VVVNFAPTIQEIK,NEGR1.WSVDPR,NELL2.AFLFQDTPR,NELL2.FTGSSWIK,NELL2.SALAYVDGK,NEO1.DVVASLVSTR,NEUS.ALGITEIFIK,NEUS.QEVPLATLEPLVK,NGF.SAPAAAIAAR,NICA.ALADVATVLGR,NICA.APDVTTLPR,NP

In [15]:
#check dataset statistics
data.describe()

,A1AT.AVLTIDEK,A1AT.LSITGTYDLK,A1AT.SVLGQLGITK,A1BG.NGVAQEPVHLDSPAIK,A1BG.SGLSTGWTQLSK,A2GL.DLLLPQPDLR,A2GL.VAAGAFQGLR,A4.LVFFAEDVGSNK,A4.THPHFVIPYR,A4.WYFDVTEGK,AACT.ADLSGITGAR,AACT.EIGELYLPK,AACT.NLAVSQVVHK,AATC.IVASTLSNPELFEEWTGNVK,AATC.LALGDDSPALK,AATC.NLDYVATSIHEAVTK,AATM.FVTVQTISGTGALR,AFAM.DADPDTFFAK,AFAM.FLVNLVK,AFAM.LPNNVLQEK,ALDOA.ALQASALK,ALDOA.QLLLTADDR,AMBP.AFIQLWAFDAVK,AMBP.ETLLQDFR,AMBP.FLYHK,AMD.IPVDEEAFVIDFKPR,AMD.IVQFSPSGK,AMD.NGQWTLIGR,APLP2.HYQHVLAVDPEK,APLP2.WYFDLSK,APOB.IAELSATAQEIIK,APOB.SVSLPSLDPASAK,APOB.TGISPLALIK,APOD.VLNQELR,APOE.AATVGSLAGQPLQER,APOE.CLAVYQAGAR,APOE.LAVYQAGAR,APOE.LGADMEDVR,APOE.LGPLVEQGR,B2MG.VEHSDLSFSK,B2MG.VNHVTLSQPK,B3GN1.EPGEFALLR,B3GN1.TALASGGVLDASGDYR,B3GN1.YEAAVPDPR,BACE1.SIVDSGTTNLR,BASP1.ETPAATEAPSSTPK,BTD.LSSGLVTAALYGR,BTD.SHLIIAQVAK,C1QB.LEQGENVFLQATDK,C1QB.VPGLYYFTYHASSR,CA2D1.FVVTDGGITR,CA2D1.IKPVFIEDANFGR,CA2D1.TASGVNQLVDIYEK,CAD13.DIQGSLQDIFK,CAD13.INENTGSVSVTR,CAD13.YEVSSPYFK,CADM3.EGSVPPLK,CADM3.GNPVPQQYLWEK,CADM3.SLVTVLGIPQKPIITGYK,CAH1.VLDALQAIK,CAH1.YSSLAEAASK,CATA.LFAYPDTHR,CATD.LVDQNIFSFYLSR,CATD.VSTLPAITLK,CATD.YSQAVPAVTEGPIPEVLK,CATL1.VFQEPLFYEAPR,CCKN.AHLGALLAR,CCKN.NLQNLDPSHR,CD14.AFPALTSLDLSDNPGLGER,CD14.FPAIQNLALR,CD14.SWLAELQQWLKPGLK,CD59.AGLQVYNK,CERU.IYHSHIDAPK,CERU.NNEGTYYSPNYNPQSR,CFAB.DAQYAPGYDK,CFAB.VSEADSSNADWVTK,CFAB.YGLVTYATYPK,CH3L1.ILGQQVPYATK,CH3L1.SFTLASSETGVGAPISGPGIPGR,CH3L1.VTIDSSYDIAK,CLUS.IDSLLENDR,CLUS.SGSGLVGR,CLUS.VTTVASHTSDSDVPSGVTEVVVK,CMGA.EDSLEAGLPLQVR,CMGA.SEALAVDGAGKPGAEEAQDPEGK,CMGA.SGEATDGARPQALPEPMQESK,CMGA.SGELEQEEER,CMGA.YPGPQAEGDSEGLSQGLVDR,CNDP1.ALEQDLPVNIK,CNDP1.VFQYIDLHQDEFVQTLK,CNDP1.WNYIEGTK,CNTN1.DGEYVVEVR,CNTN1.TTKPYPADIVVQFK,CNTN2.IIVQAQPEWLK,CNTN2.TTGPGGDGIPAEVHIVR,CNTN2.VIASNILGTGEPSGPSSK,CNTP2.HELQHPIIAR,CNTP2.VDNAPDQQNSHPDLAQEEIR,CNTP2.YSSSDWVTQYR,CO2.DFHINLFR,CO2.HAIILLTDGK,CO2.SSGQWQTPGATR,CO3.IHWESASLLR,CO3.LSINTHPSQKPLSITVR,CO3.TELRPGETLNVNFLLR,CO3.TGLQEVEVK,CO3.VPVAVQGEDTVQSLTQGDGVAK,CO4A.DHAVDLIQK,CO4A.GSFEFPVGDAVSK,CO4A.LGQYASPTAK,CO4A.NVNFQK,CO4A.VLSLAQEQVGGSPEK,CO4A.VTASDPLDTLGSEGALSPGGVASLLR,CO5.DINYVNPVIK,CO5.TLLPVSKPEIR,CO5.VFQFLEK,CO6.ALNHLPLEYNSALYSR,CO6.SEYGAALAWEK,CO8B.IPGIFELGISSQSDR,CO8B.SDLEVAHYK,CO8B.YEFILK,COCH.GVISNSGGPVR,CRP.ESDTSYVSLK,CSTN1.GNLAGLTLR,CSTN1.IHGQNVPFDAVVVDK,CSTN1.IPDGVVSVSPK,CSTN3.ATGEGLIR,CSTN3.ESLLLDTTSLQQR,CUTA.TQSSLVPALTDFVR,CYTC.ALDFAVGEYNK,DAG1.GVHYISVSATR,DAG1.LVPVVNNR,DAG1.VTIPTDLIASSGDIIK,DIAC.ATYIQNYR,ENOG.GNPTVEVDLYTAK,ENOG.LGAEVYHTLK,ENPP2.SYPEILTLK,ENPP2.WWGGQPLWITATK,EXTL2.VIVVWNNIGEK,FABPH.SIVTLDGGK,FABPH.SLGVGFATR,FAM3C.GINVALANGK,FAM3C.SALDTAAR,FAM3C.SPFEQHIK,FAM3C.TGEVLDTK,FBLN1.AITPPHPASQANIIFDITEGNLR,FBLN1.IIEVEEEQEDPYLNDR,FBLN1.TGYYFDGISR,FBLN3.IPSNPSHR,FBLN3.LTIIVGPFSF,FBLN3.SGNENGEFYLR,FETUA.AHYDLR,FETUA.FSVVYAK,FETUA.HTLNQIDEVK,FMOD.YLPFVPSR,GFAP.ALAAELNQLR,GOLM1.DQLVIPDGQEEEQEAAGEGR,GOLM1.QQLQALSEPQPR,GRIA4.EYPGSETPPK,GRIA4.LQNILEQIVSVGK,GRIA4.NTDQEYTAFR,HBA.FLASVSTVLTSK,HBA.TYFPHFDLSHGSAQVK,HBA.VGAHAGEYGAEALER,HBB.EFTPPVQAAYQK,HBB.SAVTALWGK,HBB.VNVDEVGGEALGR,HEMO.NFPSPVDAAFR,HEMO.QGHNSVFLIK,HEMO.SGAQATWTELPWPHEK,I18BP.LWEGSTSR,IBP2.HGLYNLK,IBP2.LIQGAPTIR,IGSF8.DTQFSYAVFK,IGSF8.LQGDAVVLK,IGSF8.VVAGEVQVQR,ITIH1.EVAFDLEIPK,ITIH1.QYYEGSEIVVAGR,ITIH5.SYLEITPSR,KAIN.FYYLIASETPGK,KAIN.LGFTDLFSK,KAIN.VGSALFLSHNLK,KAIN.WADLSGITK,KLK10.ALQLPYR,KLK11.LPHTLR,KLK6.ESSQEQSSVVR,KLK6.LSELIQPLPLER,KLK6.YTNWIQK,KNG1.DIPTNSPELEETLTHTITK,KNG1.QVVAGLNFR,KNG1.TVGSDTFYSFK,KPYM.LDIDSPPITAR,L1CAM.AQLLVVGSPGPVPR,L1CAM.LVLSDLHLLTQSQVR,L1CAM.WRPVDLAQVK,LAMB2.AQGIAQGAIR,LPHN1.LVVSQLNPYTLR,LPHN1.SGETVINTANYHDTSPYR,LRC4B.HLEILQLSK,LRC4B.LTTVPTQAFEYLSK,LTBP2.EQDAPVAGLQPVER,MIME.ESAYLYAR,MIME.ETVIIPNEK,MIME.LEGNPIVLGK,MOG.VVHLYR,MUC18.EVTVPVFYPTEK,MUC18.GATLALTQVTPQDER,NBL1.LALFPDK,NCAM1.AGEQDATIHLK,NCAM1.GLGEISAASEFK,NCAM2.ASGSPEPAISWFR,NCAM2.IIELSQTTAK,NCAN.APVLELEK,NCAN.LSSAIIAAPR,NEGR1.SSIIFAGGDK,NEGR1.VVVNFAPTIQEIK,NEGR1.WSVDPR,NELL2.AFLFQDTPR,NELL2.FTGSSWIK,NELL2.SALAYVDGK,NEO1.DVVASLVSTR,NEUS.ALGITEIFIK,NEUS.QEVPLATLEPLVK,NGF.SAPAAAIAAR,NICA.ALADVATVLGR,NICA.APDVTTLPR,NP

## Map Omics dataset to treemap images

First, filter out low values lower than a threshold

In [16]:
# threshold = -3
# for col in data:
#     data[col] = data.loc[data[col]>-3]
    
# print(data.shape)

(135, 320)


Then, match the gene names with KEGG-IDs and select the genes with the highest average expression values for the KEGG-IDs corresponding to multiple genes in the data matrix